**车站信息**

StationReference = Name | MayBeInitial | Platforms

|名称 | 是否可初始化? | 站台 |  
 |--|--|--|  
|a = 武汉站高速场 | 1 | 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14 
b = 京广高速咸宁北方向 | 1 | 1, 2 
 c = 武汉动车所 | 1 | 3, 4, 2, 1 
 d = 武九客专葛店南方向 | 1 | 2, 1 
 e = 京广高速孝感北方向 | 1 | 2, 1 
 f = 沪蓉线红安西方向 | 1 | 1, 2 
 g = 滠武线滠口方向 | 1 | 2, 1 
 h = 沪蓉线汉口方向 | 1 | 2, 1 
 i = 丹水池联络线汉口方向 | 1 | 2, 1 
 j = 至京广线 | 1 | 2, 1 
 k = 武汉站货车外绕线 | 1 | 2, 1 
 l = 武汉南线路所 | 1 |  
 m = 横店东 | 1 |  
 n = L1L2线路所 | 1 |  

**时刻表**  
 
**例子**  
|列车编号|类型|最高时速|动拖布置|flag|车站1|车站2|车站3|   
|--|--|--|--|--|--|--|--|  
|G3472 |COMMUTER |300 |LPPLLPPL |X1 :| b#2#21:40:00#0| e#0#21:45:00#30 |a#0#22:19:00#0|  




**车次相关信息**  
车次信息不变 默认commuter  
依照类型设置最高速度 D->200 G->300....  
车辆细节 先默认 MTTM 后续修改长编或者重连?  
  

**停站相关信息**  
仅三点式 进场车站 停站 离场车站  
始发终到则进行利用抵达时刻等替换  
公式=IF(B2<>$J$2,B2,TEXT(E2-0.5/24,"h:mm:ss"))    
车站 筛选所有车次始发终到，设置字典更改进场离场   

  
**表格信息**  
爬虫以及selenium自动化获取数据   
二者合并为一张工作表并预处理为上图格式  
始发站和终到站更换为进场车站和离场车站  
  

**车站信息**  
由于自动化需要,武汉站分为京广高速场和综合场
基于ORM大小略作修改,仅保留联络线部分  

  

**库函数部分及常规列车信息部分**  
- 导入库函数，处理表格以及替换字符并导出  
- 使用字典建立常规信息的映射关系  

In [1]:
import numpy
import pandas
import datetime
from random import randint

from selenium import webdriver
from selenium.webdriver.common.keys import Keys  # 引用keys包
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time


Excelpath = "武汉站.xlsx"  # Excel时刻表文件路径
TextPath = "train.txt"  # 游戏时刻表文件路径
# 速度和编组以及类型映射关系,0为普速1为动车2为高速
species = {'K': ['120', 'LPPPPPP', 0], 'T': ['140', 'LPPPPPP', 0], 'Z': ['160', 'LPPPPPP', 0],
           'D': ['200', 'LPPL', 1], 'C': ['200', 'LPPL', 1], 'G': ['300', 'LPPLLPPL', 2]}


# 车站-编号,掉向,用时以及运行车辆种类映射关系
# 图片左(0)右(1)侧线路key值相同则掉向,
# 国铁车辆行走左侧,2为数据为左侧股道编号
# [车站编号,车站所在侧(0为左侧),车辆进场股道,车辆离场行走股道,到达中心车站所用时间,行走车辆类型]
station = {'武汉站': ['a', -1, 0, 0, 0, 'CDG'], '武汉动车所': ['c', 1, 0, 0, 5, 'GDC'],
           '京广高速咸宁北方向': ['b', 1, 2, 1, 5, 'G'], '武九客专葛店南方向': ['d', 1, 2, 1, 4, 'GD'],
           '京广高速孝感北方向': ['e', 1, 1, 2, 10, 'G'], '沪蓉线红安西方向': ['f', 0, 1, 2, 10, 'CDG'],
           '沪蓉线汉口方向': ['h', 0, 1, 2, 5, 'G'], '丹水池联络线汉口方向': ['i', 0, 1, 2, 7, 'CD'], }

ThisStation="武汉站"
# track = {}
# 车型关系--待筛选，默认短编
marshalling = {}

# 始发终到映射关系--待筛选
dst = {}

# 股道到达,图片左右侧线路,,key值不同则掉向
# turnst = {'武九客专武汉方向': 0, '武九客专鄂州方向': 1, '武冈城际黄冈方向': 1}

trainDF = pandas.DataFrame(
    columns=['车次', '类型', '最高时速', '动拖布置', 'flag'])  # 列车整体信息
arriveStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息
stopStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 停站信息
leaveStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 离场信息

# 存储字符串形式的最终结果
trainList = []
arriveStList = []
stopStList = []
leaveStList = []


**读取表格及数据处理部分**  
- 1,读取表格并去除空行  
- 2,从车次提取速度等级  
- 3,筛选始发站终到站建立替换字典  
  
  
**车次字符串生成部分**  
1,车次信息部分  
 - 车次号,类型,动拖布置及flag保持默认  
 - 速度等级依照KTZDG等区分映射  

2,停站部分    
 - 停站2(主要车站) 到时及停站时间来自表格部分,股道由由车站大屏生成,终到车则为任意股道  
 - 进场及离场部分 依照游戏先期测试进行平移推算,股道依照国铁车辆靠左行驶安排,  

In [2]:
# 读取文件
sheet0 = pandas.read_excel(io=Excelpath, sheet_name="武汉站")  # 车次及始发终到和检票口信息

# 去除空行
#sheet0 = sheet0.dropna()
sheet0 = sheet0[~(sheet0["车次"].isnull())]  # 删掉空行

sheet = sheet0
sheet = sheet.reset_index(drop=True)  # 按照车次信息进行连接，合并检票口sheet1
sheet = sheet.drop_duplicates(subset="车次")
# sheet.replace("--:--",numpy.nan())
# sheet=sheet.fillna(value=0)

print(sheet.head())

trainInfo = sheet["车次"].to_frame()  # 车辆信息
# 统计始发终到车站信息
ts = pandas.concat([sheet0["始发站"].value_counts(),
                   sheet0["终到站"].value_counts()])
totalStation = ts.index

# print(set(list(totalStation)))  # 所有始发站和终到站统计


      车次  类型 始发站  终到站        到时        开时    检票口
0  D7001  动车  武汉  黄冈东  05:37:00  06:07:00  检票口A7
1  G3220  高速  武汉  郑州东  05:50:00  06:20:00  检票口B2
2  D2179  动车  武汉   安庆  05:53:00  06:23:00  检票口A7
3  C5511  动车  武汉  大冶北  05:56:00  06:26:00  检票口A5
4  C5521  动车  武汉  大冶北  06:03:00  06:33:00  检票口A6


**始发终到替换**  
存在多路径问题  
- 依照速度等级来进行不同选择
- 特殊的则还是手动替换

In [3]:
# 手动建立映射关系,别的方法不会

def getArrLeaSt(type, station):
    speedType = type[0]
    stcode = ""
    # 多路径车站部分
    if speedType == "G":  # 高速
        ArrLeaveSt = {
            '汉口': '沪蓉线汉口方向',
            '宜昌东': '沪蓉线汉口方向',
            '襄阳东': '沪蓉线汉口方向',
            '成都东': '沪蓉线汉口方向',
            '利川': '沪蓉线汉口方向',
            '重庆北': '沪蓉线汉口方向',
        }
    elif speedType == "D" or speedType == "C":  # 动车城际
        ArrLeaveSt = {          
            '汉口': '丹水池联络线汉口方向',
            '宜昌东': '丹水池联络线汉口方向',
            '襄阳东': '丹水池联络线汉口方向',
            '成都东': '丹水池联络线汉口方向',
            '利川': '丹水池联络线汉口方向',
            '重庆北': '丹水池联络线汉口方向',
        }
    else:  # 普速
        ArrLeaveSt = {

        }

    stcode = ArrLeaveSt.get(station)
    if stcode != None:
        # print("车次:{0} 车站:{1} ".format(type, station))
        return stcode

    # 常规单路径车站
    ArrLeaveSt = {
        '武汉':'武汉动车所',
        '南宁东': '京广高速咸宁北方向',
        '北京西': '京广高速孝感北方向',
        '南昌西': '武九客专葛店南方向',
        '香港西九龙': '京广高速咸宁北方向',
        '广州南': '京广高速咸宁北方向',
        '武汉': '武汉动车所',
        '深圳北': '武九客专葛店南方向',
        '西安北': '京广高速孝感北方向',
        '温州南': '武九客专葛店南方向',
        '兰州西': '京广高速孝感北方向',
        '长沙南': '京广高速咸宁北方向',
        '郑州东': '京广高速孝感北方向',
        '贵阳北': '京广高速咸宁北方向',
        '昆明南': '京广高速咸宁北方向',
        '上海虹桥': '沪蓉线红安西方向',
        '合肥东': '沪蓉线红安西方向',
        '厦门': '武九客专葛店南方向',
        '南京南': '沪蓉线红安西方向',
        '福州': '武九客专葛店南方向',
        '杭州东': '武九客专葛店南方向',
        '南昌': '武九客专葛店南方向',
        '济南': '京广高速孝感北方向', 
        '青岛北': '京广高速孝感北方向',
        '沈阳北': '京广高速孝感北方向',
        '厦门北': '武九客专葛店南方向',
        '大冶北': '武九客专葛店南方向',
        '福州南': '武九客专葛店南方向',
        '黄冈东': '武九客专葛店南方向',
        '安庆': '武九客专葛店南方向',
    }
    stcode = ArrLeaveSt.get(station)
    if stcode != None:
        return stcode
    else:
        print("车次:{0} 车站:{1} 未找到".format(type, station))
        exit()

def checkin(entrance):
    # 终到车
    if entrance==0:
        return 0

    # 检票口A5 非终到车
    en= entrance[-1]
    if en=='1': #高速场
        return randint(1,3)
    elif en=='2':
        return randint(4,6)
    elif en=='3':
        return randint(7,8)
    elif en=='4':
        return randint(9,11)
    elif en=='5':
        return randint(11,13)
    elif en=='6':
        return randint(13,15)
    elif en=='7':#综合场
        return randint(16,18)
    else:
        return randint(18,20)
    



In [4]:
# 生成车次信息
# '列车编号','类型','最高时速','动拖布置','flag' 列车整体信息

for index, row in trainInfo.iterrows():
    trainNum = row["车次"]
    maxspeed = (species.get(trainNum[0]))[0]  # 有字头的列车
    if maxspeed == None:  # 最高速度未找到
        maxspeed = '120'  # 普客

    marshalling = (species.get(trainNum[0]))[1]
    # trainTC=marshalling.get() #动拖布置
    trainDF.loc[index] = [trainNum, "COMMUTER", maxspeed, marshalling, "X1"]
    trainList.append("{0} {1} {2} {3} {4} : ".format(
        trainNum, "COMMUTER", maxspeed, marshalling, "X1"))
trainDF.head()


,车次,类型,最高时速,动拖布置,flag
0,D7001,COMMUTER,200,LPPL,X1
1,G3220,COMMUTER,300,LPPLLPPL,X1
2,D2179,COMMUTER,200,LPPL,X1
3,C5511,COMMUTER,200,LPPL,X1
4,C5521,COMMUTER,200,LPPL,X1


In [5]:
mainstation="武汉"

wd = webdriver.Firefox()
wd.implicitly_wait(10)
wd.get('https://www.12306.cn/index/view/infos/ticket_check.html')

def getEntance(wd,mainstation,checi):
    #输入车次并确认
    element1 = wd.find_element(By.XPATH, '//*[@id="ticket_check_trainNum"]')

    element1.clear()
    element1.send_keys(checi+'\n')
    time.sleep(1)
    element1.send_keys(Keys.ENTER) #回车键Enter

    #点击下拉框以获得停站选项
    time.sleep(1)
    wd.find_element(By.XPATH,'//*[@id="ticketEntranceSel"]').click()

    #选择停站
    time.sleep(1) 
    elements = wd.find_elements(By.CSS_SELECTOR, '.model-select-option > li')
    for ele in elements: #在所有停站中遍历信息
        if ele.text==station:
            ele.click() #点击下拉框对应选项确认
            #如果查询到
            btn=wd.find_element(By.CSS_SELECTOR,".btn")
            btn.click()      #点击查询按钮
            checkin=wd.find_elements(By.CSS_SELECTOR,".check-numnew")  
            time.sleep(0.5)
            entrance=checkin[0].text #得到检票口文本信息
            # print("车次:{0},检票口:{1}".format (checi,entrance))
            if entrance!=None:
                return entrance
    #没有查询到则为终到车次
    print("车次:{0} ,{1}站无检票口信息".format (checi,station))
    return 0





In [6]:
# 生成停站信息
for index, row in sheet.iterrows():
    arriveSt = getArrLeaSt(row["车次"], row["始发站"])
    leaveSt = getArrLeaSt(row["车次"], row["终到站"])

    # 上下行编号区分车辆进场股道为1,2道,进场及离场股道
    # 靠左行走
    arriveTrack = station.get(arriveSt)[2]
    leaveTrack = station.get(leaveSt)[3]

    # 停站股道,依照检票口选择,没有则填为0，任意股道
    # 使用int()去掉float类型的.0,防止因为格式问题读不出股道
    row = row.fillna(0)
    stopTrack = checkin(row["检票口"])

    at = row["到时"]

    lt = row["开时"]

    # 统一时间格式

    strTime1 = datetime.datetime.strptime(str(at), "%H:%M:%S")
    strTime2 = datetime.datetime.strptime(str(lt), "%H:%M:%S")

    stopTime = (strTime2-strTime1).seconds/60  # 分钟为单位的停站时间

    # 时间处理部分,由到达时刻推出进场时刻
    arrTime1 = strTime1-datetime.timedelta(minutes=(station.get(arriveSt))[4])
    arrTime2 = str(arrTime1.strftime('%Y-%m-%d %H:%M:%S'))[-8:]  # 仅保留时分秒
    arrTime = datetime.datetime.strptime(arrTime2, "%H:%M:%S")  # 转换格式

    # 离场时刻
    leaveTime1 = strTime2+datetime.timedelta(minutes=(station.get(leaveSt))[4])
    leaveTime2 = str(leaveTime1.strftime('%Y-%m-%d %H:%M:%S'))[-8:]
    leaveTime = datetime.datetime.strptime(leaveTime2, "%H:%M:%S")

    # 进场
    arriveStDF.loc[index] = [arriveSt, arriveTrack, arrTime2, 0]
    arriveStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(arriveSt))[0], arriveTrack, arrTime2, 0))
    # 停站
    stopStDF.loc[index] = [ThisStation, stopTrack, row["到时"], stopTime]
    stopStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(ThisStation))[0], stopTrack, row["到时"], int(stopTime)))

    # 离场
    leaveStDF.loc[index] = [leaveSt, leaveTrack, leaveTime2, 0]
    leaveStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(leaveSt))[0], leaveTrack, leaveTime2, 0))
    


In [7]:
print(arriveStDF.head(), '\n', stopStDF.head(), '\n', leaveStDF.head())  # 部分信息
wd.quit()

    车站名称  股道      到达时间  停站时间
0  武汉动车所   0  05:32:00     0
1  武汉动车所   0  05:45:00     0
2  武汉动车所   0  05:48:00     0
3  武汉动车所   0  05:51:00     0
4  武汉动车所   0  05:58:00     0 
   车站名称  股道      到达时间  停站时间
0  武汉站  17  05:37:00  30.0
1  武汉站   4  05:50:00  30.0
2  武汉站  17  05:53:00  30.0
3  武汉站  12  05:56:00  30.0
4  武汉站  15  06:03:00  30.0 
         车站名称  股道      到达时间  停站时间
0  武九客专葛店南方向   1  06:11:00     0
1  京广高速孝感北方向   2  06:30:00     0
2  武九客专葛店南方向   1  06:27:00     0
3  武九客专葛店南方向   1  06:30:00     0
4  武九客专葛店南方向   1  06:37:00     0


**最终车次结果**  
合并车辆信息和停站信息  
  
**导出部分**  
导出为train.txt手动附加原文件头部之后替换原时刻表文件  

In [8]:
timeTable = open(file=TextPath, mode="w")  # 覆盖写入txt
finalRes = []
for i in range(0, len(arriveStList)):
    tstr = "{0}{1} {2} {3} ".format(
        trainList[i], arriveStList[i], stopStList[i], leaveStList[i])
    finalRes.append(tstr)  # 用于展示的列表
    tstr1 = tstr+'\n'
    timeTable.writelines(tstr1)  # 按行写入

timeTable.close()
finalRes


['D7001 COMMUTER 200 LPPL X1 : c#0#05:32:00#0 a#17#05:37:00#30 d#1#06:11:00#0 ',
 'G3220 COMMUTER 300 LPPLLPPL X1 : c#0#05:45:00#0 a#4#05:50:00#30 e#2#06:30:00#0 ',
 'D2179 COMMUTER 200 LPPL X1 : c#0#05:48:00#0 a#17#05:53:00#30 d#1#06:27:00#0 ',
 'C5511 COMMUTER 200 LPPL X1 : c#0#05:51:00#0 a#12#05:56:00#30 d#1#06:30:00#0 ',
 'C5521 COMMUTER 200 LPPL X1 : c#0#05:58:00#0 a#15#06:03:00#30 d#1#06:37:00#0 ',
 'D5818 COMMUTER 200 LPPL X1 : c#0#06:01:00#0 a#5#06:06:00#30 i#2#06:43:00#0 ',
 'G1103 COMMUTER 300 LPPLLPPL X1 : c#0#06:18:00#0 a#5#06:23:00#30 b#1#06:58:00#0 ',
 'D7019 COMMUTER 200 LPPL X1 : c#0#06:25:00#0 a#11#06:30:00#30 d#1#07:04:00#0 ',
 'G258 COMMUTER 300 LPPLLPPL X1 : c#0#06:28:00#0 a#5#06:33:00#30 e#2#07:13:00#0 ',
 'G431 COMMUTER 300 LPPLLPPL X1 : c#0#06:31:00#0 a#10#06:36:00#30 b#1#07:11:00#0 ',
 'G2626 COMMUTER 300 LPPLLPPL X1 : c#0#06:33:00#0 a#3#06:38:00#30 e#2#07:18:00#0 ',
 'D7003 COMMUTER 200 LPPL X1 : c#0#06:40:00#0 a#20#06:45:00#30 d#1#07:19:00#0 ',
 'G3237 COMMUTE